# Multilabel Classificator
The results obtained with the CNN models for just one label classification where is not great enough, maybe de cause of those results is that some songs are not only classify as one genre, it could be in various. Due to this  possible reason, this notebook has been created.

## Initialisation

### Libraries

In [9]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Input, Dropout, Add
from tensorflow.keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import MaxNorm
from keras.optimizers import SGD, Adam
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import os
import ast
from pathlib import Path

# Import function to plot the results
import plots

### Data Configuration Parameters
Configuration variables related to the data

In [10]:
# Randomize the initial network weights
random_seed = True

# Paths to where training, testing, and validation images are
database_dir = 'dataset_multilabel'
train_dir = f'{database_dir}/training/spectrogram'
val_dir = f'{database_dir}/val/spectrogram'
test_dir = f'{database_dir}/test/spectrogram'

# Directory where to store weights of the model and results
root_dir = "results"
# Create root directory for results if it does not exist
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Input dimension (number of subjects in our problem)
num_classes = 6

# Name of each gesture of the database
# CLASSES = [x for x in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, x))]
CLASSES = ['Alternative', 'Pop', 'Rock', 'Dance', 'Classical', 'Techno']
print(f'The classess to classify are: {CLASSES}')

# Parameters that characterise the spectrogram
img_height = 369
img_width = 496
img_channels = 3
color_mode = 'rgb'

The classess to classify are: ['Alternative', 'Pop', 'Rock', 'Dance', 'Classical', 'Techno']


### Configuration Training Parameters

In [11]:
# Parameters that configures the training process
batch_size = 1  # Batch size
epochs = 5  # Number of epochs
initial_lr = 1e-2   # Learning rate
seed = 42  # Random number
modelCNN = 'LSTM'  # RNN model which will be used
version = f'BS{batch_size}_E{epochs}_LR{initial_lr}'
experiment_dir = f'{root_dir}/{modelCNN}'

# Create experiment directory if it does not exist
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

# Set random seed
if random_seed:
    seed = np.random.randint(0,2*31-1)
else:
    seed = 5
np.random.seed(seed)
tf.random.set_seed(seed)

### Loading of training, validation and test datasets of images
1. Dataframes with multiabel info
2. Training dataset
3. Validation dataset
4. Test dataset

In [12]:
# mlb = MultiLabelBinarizer()

# Training labels
train_df = pd.read_csv(f'{database_dir}/training_multilabel.csv', delimiter='\t')
train_df['Genres']= train_df['Genres'].apply(lambda x:x.split(", "))
# train_df['Genres'] = mlb.fit_transform(train_df['Genres'])
train_df['TRACK_ID'] = train_df['TRACK_ID'].astype(str)

# Validation labels
val_df = pd.read_csv(f'{database_dir}/validation_multilabel.csv', delimiter='\t')
val_df['Genres']= val_df['Genres'].apply(lambda x:x.split(", "))
val_df['TRACK_ID'] = val_df['TRACK_ID'].astype(str)

# Test labels
test_df = pd.read_csv(f'{database_dir}/test_multilabel.csv', delimiter='\t')
test_df['Genres'] = test_df['Genres'].apply(lambda x:x.split(", "))
test_df['TRACK_ID'] = test_df['TRACK_ID'].astype(str)

In [13]:
train_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(img_width,img_height))

val_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=val_df,
    directory=val_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(img_width,img_height))

test_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=False,
    class_mode=None,
    target_size=(img_width,img_height))

# x_generator = ImageDataGenerator().flow_from_dataframe(
#     dataframe=train_df.tail(10),  # Seleccionar las últimas 10 filas del DataFrame
#     directory=train_dir,
#     x_col='TRACK_ID',
#     y_col='Genres',
#     batch_size=batch_size,
#     seed=seed,
#     shuffle=True,
#     class_mode='categorical',
#     classes=CLASSES
# )
# 
# # Obtener una muestra de lotes del generador de entrenamiento
# sample_batch = next(x_generator)
# 
# # Imprimir información sobre las columnas X e y del lote de muestra
# print("\nLote de muestra:")
# print("Columna X (features):")
# print(sample_batch[0])  # Este será el lote de imágenes (X)
# print("\nColumna y (etiquetas):")
# print(sample_batch[1])  # Este será el lote de etiquetas (y)

Found 1186 validated image filenames belonging to 6 classes.
Found 204 validated image filenames belonging to 6 classes.
Found 204 validated image filenames.


/Users/agv/opt/anaconda3/envs/DLMT/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 14 invalid image filename(s) in x_col="TRACK_ID". These filename(s) will be ignored.
  warnings.warn(


## Training process
#### Available Models: CNN

### Random model

In [14]:
def build_model(img_width,img_height,img_channels):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_width,img_height,img_channels)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))
    return model

### VGG-16

In [15]:
def VGG16(img_width,img_height,img_channels):
    #  dropout rate for FC layers
    dropout=0.5

    # CNN architecture
    input_image = Input(shape=(img_width,img_height,img_channels))
    x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(input_image)
    x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Conv2D(256, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(256, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(256, (1, 1),padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (1, 1),padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (1, 1),padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    x1 = Flatten()(x1)

    x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x1)
    x=Dropout(dropout)(x)
    x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x)
    x=Dropout(dropout)(x)
    out= Dense(num_classes, activation='softmax')(x)

    model = Model(inputs = input_image, outputs = out)

    return model

### ResNet

In [ ]:
def RestNet(img_width,img_height,img_channels):
    dropout=0.5 # Para evitar que haya overfitting cancela algunas conexiones entre las neuronas aleatoriamente.

    # CNN architecture
    input_image = Input(shape=(img_width,img_height,img_channels))
    x0 = Conv2D(64, (3, 3),padding='same', activation='relu')(input_image)
    x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(x0)
    x1 = MaxPooling2D((2, 2))(x1)
    x2 = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
    x3 = Conv2D(128, (3, 3),padding='same')(x2)
    x4 = Add()([x3, x2])
    x4 = Activation('relu')(x4)
    x4 = MaxPooling2D((2, 2))(x4)
    x4 = Conv2D(256, (3, 3),padding='same', activation='relu')(x4)
    x5 = Conv2D(256, (3, 3),padding='same', activation='relu')(x4)
    x5 = Conv2D(256, (1, 1),padding='same')(x5)
    x6 = Add()([x5, x4])
    x6 = Activation('relu')(x6)
    x6 = MaxPooling2D((2, 2))(x6)
    x6 = Conv2D(512, (3, 3),padding='same', activation='relu')(x6)
    x7 = Conv2D(512, (3, 3),padding='same', activation='relu')(x6)
    x7 = Conv2D(512, (1, 1),padding='same')(x7)
    x8 = Add()([x7, x6])
    x8 = Activation('relu')(x8)
    x8 = MaxPooling2D((2, 2))(x8)
    x8 = Conv2D(512, (3, 3),padding='same', activation='relu')(x8)
    x9 = Conv2D(512, (3, 3),padding='same', activation='relu')(x8)
    x9 = Conv2D(512, (1, 1),padding='same')(x9)
    x10 = Add()([x9, x8])
    x10 = Activation('relu')(x10)
    x10 = MaxPooling2D((2, 2))(x10)
    x10 = Flatten()(x10)

    x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x10)
    x=Dropout(dropout)(x)
    x=Dense(4096, activation='relu', kernel_constraint=MaxNorm(3))(x)
    x=Dropout(dropout)(x)
    out= Dense(num_classes, activation='softmax')(x)

    model = Model(inputs = input_image, outputs = out);

    return model

## Model execution

In [16]:
#Model
model = ''
if modelCNN == 'Random_Model':
    model = build_model(img_width,img_height,img_channels)
elif modelCNN == 'VGG':
    model = VGG16(img_width,img_height,img_channels)
elif modelCNN == 'ResNet':
    model = RestNet(img_width,img_height,img_channels)
else:
    print('Wrong model selection or Model no available\n')

# Print the architecture of the model
model.summary()

2024-01-14 23:48:57.699603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 496, 369, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 496, 369, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 496, 369, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 248, 184, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 248, 184, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 248, 184, 128)     147584    
                                                             

## Set model training process
#### Configuration of several training decisions:
1. Optimizer using `Adam`
2. Model training configuration using `compile` with `binary_crossentropy` due to the classification labeling

In [17]:
# 1. Configure optimizer
adam = Adam(learning_rate=initial_lr)

# 2. Configure training process
model.compile(loss = ['binary_crossentropy'],optimizer=adam, metrics=['accuracy'])

## Train the model
1. Load parameters from previous trainings if they exist.
2. Fit the model
3. Save the weights

In [18]:
# Load pretrained model
weights_path = f"weights_{version}.h5" # Name of the file to store the weights
weights_file = Path(weights_path)
weights_load_path = f'{experiment_dir}/{weights_path}'
if weights_load_path:
    try:
        model.load_weights(weights_load_path)
        print("Loaded model from {}".format(weights_load_path))
    except:
        print("Impossible to find weight path. Returning untrained model")

# Fit the model
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, batch_size=batch_size)

# Save weights
weights_save_path = os.path.join(experiment_dir, weights_path)
model.save_weights(weights_save_path)

Impossible to find weight path. Returning untrained model
Epoch 1/5
  57/1186 [>.............................] - ETA: 1:18:45 - loss: 7880324096.0000 - accuracy: 0.1754

KeyboardInterrupt: 

## Training Results
Accuracy and Loss obtained along the training process

In [ ]:
plots.accloss(history, modelCNN, experiment_dir, version)

## Testing
### Model Testing
1. Compute the loss function and accuracy for the test data
2. Confusion Matrix obtained from testing results

In [ ]:
# X_test
X_test = test_generator.next()[0]
# y_test
y_test = test_generator.labels

# Evaluate model
scores = model.evaluate(x=X_test, y=y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Loss: %.2f" % scores[0])

# Obtain results to present the confusion matrix
prob_class = model.predict(x = X_test, batch_size=batch_size)
# Convert the probabilities into binary classes
threshold = 0.5 
binary_pred = tf.cast(tf.math.greater_equal(prob_class, threshold), tf.int32)
y_pred = tf.argmax(binary_pred, axis=-1)
# Classification Report
report = classification_report(y_test, y_pred, output_dict=False, target_names=CLASSES)
print(report)
# Visualize confusion matrix                                           
plots.cm_mutilabel(y_test, y_pred, modelCNN, CLASSES, experiment_dir, version)